In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random

In [2]:
df = pd.read_csv('/home/hedg/Downloads/merged.csv')
df.head()

,image_id,dx,age,sex,localization,has_hair
0,ISIC_0024698,nv,70.0,male,face,0
1,ISIC_0024693,nv,45.0,female,upper extremity,0
2,ISIC_0025964,mel,40.0,female,chest,0
3,ISIC_0030623,mel,40.0,female,chest,0
4,ISIC_0027190,mel,80.0,male,upper extremity,0


In [3]:
df.dx.value_counts()

dx
nv     22075
mel     7991
Name: count, dtype: int64

In [4]:
for i in glob.glob('/home/hedg/Downloads/HAM10000_images_part_1/*.jpg'):
    name = i[i.rfind('/')+1:]
    move_images('/home/hedg/Downloads/HAM10000_images_part_1','/home/hedg/Downloads/images', name)

In [5]:
for i in glob.glob('/home/hedg/Downloads/ISIC_2019_MEL_NV/*.jpg'):
    name = i[i.rfind('/')+1:]
    move_images('/home/hedg/Downloads/ISIC_2019_MEL_NV','/home/hedg/Downloads/images', name)

In [6]:
for i in glob.glob('/home/hedg/Downloads/MEL_hairless/*.jpg'):
    name = i[i.rfind('/')+1:]
    move_images('/home/hedg/Downloads/MEL_hairless','/home/hedg/Downloads/images', name)

In [13]:
def save_change_skin_color(img_name, image_folder, save_path_name, out_name, skin_color):
    frame = cv2.imread(image_folder+'/'+img_name + '.jpg')

    # min_YCrCb = np.array([0, 133, 77], np.uint8)
    # max_YCrCb = np.array([235, 173, 127], np.uint8)
    min_YCrCb = np.array([0, 133, 77], np.uint8)
    max_YCrCb = np.array([235, 190, 127], np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    YCrCb_converted = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
    skinMask = cv2.inRange(YCrCb_converted, min_YCrCb, max_YCrCb)
    skinMask = cv2.erode(skinMask, kernel, iterations=2)
    skinMask = cv2.dilate(skinMask, kernel, iterations=2)
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
    skinMask = cv2.bitwise_not(skinMask)
    skin = cv2.bitwise_and(frame, frame, mask=skinMask)

    skin[np.where((skin != [0, 0, 0]).all(axis=2))] = [255, 255, 255]
    frame = cv2.imread(image_folder+'/'+img_name + '.jpg')
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = cv2.bitwise_and(frame, frame, mask=skinMask)
    # skinMask = cv2.bitwise_and(frame, frame, mask=skinMask)

    desired_color = skin_color[::-1] # BGR
    desired_color = np.asarray(desired_color, dtype=np.float64)
    swatch = np.full(frame.shape, desired_color, dtype=np.uint8)
    skinMask_reverse = cv2.bitwise_not(skinMask)
    ave_color = np.array(cv2.mean(frame, mask=skinMask)[:3])


    diff_color = ave_color-desired_color
    diff_color = np.full_like(frame, diff_color, dtype=np.uint8)

    # shift input image color
    # cv2.add clips automatically
    new_img = cv2.subtract(frame, diff_color)

    # combine img and new_img using mask
    result = (frame-cv2.bitwise_and(frame, frame, mask=skinMask) + cv2.bitwise_and(new_img, new_img, mask=skinMask))
    # result = swatch

    result = result.clip(0,255).astype(np.uint8)
    cv2.imwrite(save_path_name+f'/{out_name}.jpg', result)

In [14]:
mask = pd.read_csv('/home/hedg/Downloads/mask.csv')
mask.head()

,image,apply
0,ISIC_0030623,1
1,ISIC_0027190,1
2,ISIC_0031023,0
3,ISIC_0028086,1
4,ISIC_0031177,0


In [16]:
RGB_color = [(141,85,36),
            (198,134,66),
             (200,80,50),
             (190,120,70),
             (190,120,120),
             (190,150,150),
             ]

for i, row in mask.iterrows():
    img = row.image
    if i%100 == 0:
        print(f'{i}`th index proceeded')
    if row['apply'] == 1:
        for idx, color in enumerate(RGB_color):
            save_change_skin_color(img, '/home/hedg/Downloads/images', '/home/hedg/Downloads/images', f'{img}_{idx}', color)
            df[len(df)] = row
            df[len(df)].image_id = f'{img}_{idx}'

0`th index proceeded
100`th index proceeded
200`th index proceeded
300`th index proceeded
400`th index proceeded
500`th index proceeded
600`th index proceeded
700`th index proceeded
800`th index proceeded
900`th index proceeded
1000`th index proceeded
1100`th index proceeded
1200`th index proceeded
1300`th index proceeded
1400`th index proceeded
1500`th index proceeded
1600`th index proceeded
1700`th index proceeded
1800`th index proceeded
1900`th index proceeded
2000`th index proceeded


In [38]:
def move_images(folder_from, folder_to, img_name):
    try:
        os.rename(folder_from+f'/{img_name}', folder_to+f'/{img_name}')
    except:
        print(img_name)

def move_back(folder_from, folder_to, img_name):
    os.rename(folder_from+f'/{img_name}', folder_to+f'/{img_name}')

def create_tf_dataset(image_folder, df, class_header, train_num, test_num):
    assert train_num+test_num == 1.0
    classes = df[class_header].unique()
    print('Классы:', classes)

    mel = np.array(df[df.dx == 'mel'].image_id)
    nv = np.array(df[df.dx == 'nv'].image_id)

    random.shuffle(mel)
    random.shuffle(nv)

    train_mel = mel[ : int(len(mel)*train_num)]
    mel_train_size = len(train_mel)
    train_nv = nv[ : int(mel_train_size*1.1)]

    test_mel = mel[mel_train_size : ]
    test_nv = nv[len(train_nv) : len(train_nv) + len(test_mel)]
    
    for img in train_mel:
        move_images(image_folder, '/home/hedg/Downloads/train/melanoma', img+'.jpg')
    for img in test_mel:
        move_images(image_folder, '/home/hedg/Downloads/test/melanoma', img+'.jpg')

    for img in train_nv:
        move_images(image_folder, '/home/hedg/Downloads/train/nevus', img+'.jpg')
    for img in test_nv:
        move_images(image_folder, '/home/hedg/Downloads/test/nevus', img+'.jpg')

In [39]:
create_tf_dataset('/home/hedg/Downloads/images', df, 'dx', 0.8, 0.2)

Классы: ['nv' 'mel']
ISIC_0026853.jpg
ISIC_0028442.jpg
ISIC_0029242.jpg
ISIC_0028075.jpg
ISIC_0028901.jpg
ISIC_0029163.jpg
ISIC_0032975.jpg
ISIC_0034183.jpg
ISIC_0026770.jpg
ISIC_0025313.jpg
ISIC_0029265_hairless.jpg
ISIC_0028041.jpg
ISIC_0033999.jpg
ISIC_0031711.jpg
ISIC_0031784.jpg
ISIC_0032182.jpg
ISIC_0033546.jpg
ISIC_0031779.jpg
ISIC_0033545.jpg
ISIC_0033393.jpg
ISIC_0025268.jpg
ISIC_0028760.jpg
ISIC_0032152_hairless.jpg
ISIC_0033962.jpg
ISIC_0030925.jpg
ISIC_0032888.jpg
ISIC_0033931.jpg
ISIC_0028326.jpg
ISIC_0027030.jpg
ISIC_0024400.jpg
ISIC_0033275.jpg
ISIC_0033947.jpg
ISIC_0025674.jpg
ISIC_0029698.jpg
ISIC_0030083.jpg
ISIC_0034256.jpg
ISIC_0029157.jpg
ISIC_0029909.jpg
ISIC_0033268.jpg
ISIC_0024310.jpg
ISIC_0028685.jpg
ISIC_0026236.jpg
ISIC_0030366.jpg
ISIC_0026531_hairless.jpg
ISIC_0034085_hairless.jpg
ISIC_0031586.jpg
ISIC_0027651.jpg
ISIC_0031529.jpg
ISIC_0032988.jpg
ISIC_0027956.jpg
ISIC_0028445.jpg
ISIC_0026500.jpg
ISIC_0034104.jpg
ISIC_0026105.jpg
ISIC_0028969.jpg
ISIC_002

ISIC_0033995.jpg
ISIC_0033055_hairless.jpg
ISIC_0024865_hairless.jpg
ISIC_0034262.jpg
ISIC_0032726.jpg
ISIC_0030995.jpg
ISIC_0031517.jpg
ISIC_0030970.jpg
ISIC_0024767.jpg
ISIC_0031203.jpg
ISIC_0025589.jpg
ISIC_0033885.jpg
ISIC_0026738.jpg
ISIC_0033023.jpg
ISIC_0031339.jpg
ISIC_0028103.jpg
ISIC_0029937.jpg
ISIC_0027604.jpg
ISIC_0024702_hairless.jpg
ISIC_0024865.jpg
ISIC_0029744.jpg
ISIC_0032533.jpg
ISIC_0026690.jpg
ISIC_0032637.jpg
ISIC_0034242_hairless.jpg
ISIC_0025546_hairless.jpg
ISIC_0025531.jpg
ISIC_0034253_hairless.jpg
ISIC_0032730.jpg
ISIC_0033883.jpg
ISIC_0027163.jpg
ISIC_0025904.jpg
ISIC_0029343_hairless.jpg
ISIC_0033017.jpg
ISIC_0025391.jpg
ISIC_0027149.jpg
ISIC_0032965.jpg
ISIC_0027832.jpg
ISIC_0024932.jpg
ISIC_0029318.jpg
ISIC_0034218.jpg
ISIC_0026586.jpg
ISIC_0025132.jpg
ISIC_0034062.jpg
ISIC_0031565.jpg
ISIC_0031554.jpg
ISIC_0028225.jpg
ISIC_0027407.jpg
ISIC_0034051.jpg
ISIC_0031350.jpg
ISIC_0032960.jpg
ISIC_0030521.jpg
ISIC_0033560.jpg
ISIC_0025413.jpg
ISIC_0026369.jpg
IS

ISIC_0025744.jpg
ISIC_0026120.jpg
ISIC_0032504.jpg
ISIC_0026296.jpg
ISIC_0026558.jpg
ISIC_0034222.jpg
ISIC_0024640.jpg
ISIC_0031023.jpg
ISIC_0032927_hairless.jpg
ISIC_0026113.jpg
ISIC_0033217.jpg
ISIC_0028929.jpg
ISIC_0026352.jpg
ISIC_0028022.jpg
ISIC_0029570.jpg
ISIC_0029575.jpg
ISIC_0033047.jpg
ISIC_0034002.jpg
ISIC_0033272.jpg
ISIC_0031034.jpg
ISIC_0033802.jpg
ISIC_0029272.jpg
ISIC_0029819.jpg
ISIC_0028930.jpg
ISIC_0032044.jpg
ISIC_0030423.jpg
ISIC_0027620.jpg
ISIC_0030695.jpg
ISIC_0033653.jpg
ISIC_0024999.jpg
ISIC_0031118.jpg
ISIC_0027561.jpg
ISIC_0030843.jpg
ISIC_0025234.jpg
ISIC_0031170.jpg
ISIC_0034000.jpg
ISIC_0033957.jpg
ISIC_0028584.jpg
ISIC_0029241.jpg
ISIC_0034208.jpg
ISIC_0031025.jpg
ISIC_0033559.jpg
ISIC_0028878.jpg
ISIC_0032918.jpg
ISIC_0033278.jpg
ISIC_0030060.jpg
ISIC_0024952.jpg
ISIC_0034140.jpg
ISIC_0032645.jpg
ISIC_0030883.jpg
ISIC_0024702.jpg
ISIC_0025832.jpg
ISIC_0033730.jpg
ISIC_0025451.jpg
ISIC_0032925.jpg
ISIC_0029021.jpg
ISIC_0028965.jpg
ISIC_0025766.jpg
ISIC_

ISIC_0026482.jpg
ISIC_0031790.jpg
ISIC_0029883.jpg
ISIC_0025855.jpg
ISIC_0027305.jpg
ISIC_0033530.jpg
ISIC_0026894.jpg
ISIC_0031838.jpg
ISIC_0029879.jpg
ISIC_0027773.jpg
ISIC_0033148.jpg
ISIC_0029954.jpg
ISIC_0026306.jpg
ISIC_0025918.jpg
ISIC_0028791.jpg
ISIC_0028425.jpg
ISIC_0026630.jpg
ISIC_0028374.jpg
ISIC_0033729.jpg
ISIC_0028263.jpg
ISIC_0028428.jpg
ISIC_0029365.jpg
ISIC_0026700.jpg
ISIC_0031886.jpg
ISIC_0030151.jpg
ISIC_0034295.jpg
ISIC_0027401.jpg
ISIC_0032647.jpg
ISIC_0028004.jpg
ISIC_0030084.jpg
ISIC_0025347.jpg
ISIC_0029887.jpg
ISIC_0032686.jpg
ISIC_0029667.jpg
ISIC_0029409.jpg
ISIC_0026565.jpg
ISIC_0030711.jpg
ISIC_0030495.jpg
ISIC_0031618.jpg
ISIC_0033789.jpg
ISIC_0030582.jpg
ISIC_0027032.jpg
ISIC_0031052.jpg
ISIC_0025624.jpg
ISIC_0031809.jpg
ISIC_0027377.jpg
ISIC_0028817.jpg
ISIC_0026635.jpg
ISIC_0032512.jpg
ISIC_0032904.jpg
ISIC_0027695.jpg
ISIC_0033110.jpg
ISIC_0025231.jpg
ISIC_0025430.jpg
ISIC_0027333.jpg
ISIC_0029864.jpg
ISIC_0033508.jpg
ISIC_0033094.jpg
ISIC_0028138.j

ISIC_0031247.jpg
ISIC_0026124.jpg
ISIC_0028037.jpg
ISIC_0030071.jpg
ISIC_0033625.jpg
ISIC_0030044.jpg
ISIC_0032563.jpg
ISIC_0024536.jpg
ISIC_0026812.jpg
ISIC_0024593.jpg
ISIC_0027276.jpg
ISIC_0025107.jpg
ISIC_0025181.jpg
ISIC_0029743.jpg
ISIC_0034210.jpg
ISIC_0029188.jpg
ISIC_0031360.jpg
ISIC_0034164.jpg
ISIC_0026082.jpg
ISIC_0031451.jpg
ISIC_0032980.jpg
ISIC_0033815.jpg
ISIC_0028665.jpg
ISIC_0032205.jpg
ISIC_0026033.jpg
ISIC_0027258.jpg
ISIC_0028731.jpg
ISIC_0032494.jpg
ISIC_0028296.jpg
ISIC_0033719.jpg
ISIC_0026736.jpg
ISIC_0025571.jpg
ISIC_0025740.jpg
ISIC_0033011.jpg
ISIC_0025042.jpg
ISIC_0033821.jpg
ISIC_0032607.jpg
ISIC_0026793.jpg
ISIC_0031336.jpg
ISIC_0030728.jpg
ISIC_0025464.jpg
ISIC_0024636.jpg
ISIC_0027866.jpg
ISIC_0033384.jpg
ISIC_0026320.jpg
ISIC_0027059.jpg
ISIC_0025176.jpg
ISIC_0028256.jpg
ISIC_0030648.jpg
ISIC_0031553.jpg
ISIC_0031590.jpg
ISIC_0029896.jpg
ISIC_0033914.jpg
ISIC_0034228.jpg
ISIC_0025702.jpg
ISIC_0031035.jpg
ISIC_0032667.jpg
ISIC_0025889.jpg
ISIC_0033028.j

In [36]:
def undo(image_folder):
    for img in glob.glob('/home/hedg/Downloads/train/melanoma/*'):
        move_back('/home/hedg/Downloads/train/melanoma', image_folder, img[img.rfind('/')+1:])
    for img in glob.glob('/home/hedg/Downloads/train/nevus/*'):
        move_back('/home/hedg/Downloads/train/nevus', image_folder, img[img.rfind('/')+1:])
    for img in glob.glob('/home/hedg/Downloads/test/melanoma/*'):
        move_back('/home/hedg/Downloads/test/melanoma', image_folder, img[img.rfind('/')+1:])
    for img in glob.glob('t/home/hedg/Downloads/test/nevus/*'):
        move_back('/home/hedg/Downloads/test/nevus', image_folder, img[img.rfind('/')+1:])

In [37]:
undo('/home/hedg/Downloads/images')

In [40]:
for i in glob.glob('/home/hedg/Downloads/ISIC_NEV/*.jpg'):
    name = i[i.rfind('/')+1:]
    move_images('/home/hedg/Downloads/ISIC_NEV','/home/hedg/Downloads/train/nevus', name)

In [41]:
for i in glob.glob('/home/hedg/Downloads/ISIC_MEL/*.jpg'):
    name = i[i.rfind('/')+1:]
    move_images('/home/hedg/Downloads/ISIC_MEL','/home/hedg/Downloads/train/melanoma', name)